## Reasoning with CDS Rules


We will use a logic-based reasoning engine to apply a knowledge base of rules to particular patient descriptions.

In [1]:
import pytholog as pl

### Deciding between COVID, cold, flu and allergies

Our decision support knowledge base consists of the following rules:
- fever is present in COVID and flu
- headache is present in flu, COVID and colds
- blocked nose is present in colds and allergies
- cough in COVID and flu
- shortness of breath in COVID and allergies


In [26]:
## Create a knowledge base about respiratory disease conditions
respdis = pl.KnowledgeBase("respiratory_diseases")
respdis.clear_cache()
respdis([
    "symptom(fever,covid)",
    "symptom(fever,flu)",
    "symptom(headache,flu)", 
    "symptom(headache,covid)",
    "symptom(headache,cold)", 
    "symptom(blockednose,cold)",
    "symptom(blockednose,allergy)",
    "symptom(cough,covid)",
    "symptom(cough,flu)",
    "symptom(shortbreath,covid)",
    "symptom(shortbreath,allergy)"
])

### Patient data

We have the following patients: 
- P1 has cough, fever and shortness of breath
- P2 has shortness of breath and blocked nose
- P3 has fever and headache


### Ground truth

Our patients have the following conditions:
- P1 has COVID
- P2 has allergies
- P3 has the flu

In [27]:
respdis([
    "has_symptom(p1,cough)",
    "has_symptom(p1,fever)",
    "has_symptom(p1,shortbreath)",
    "has_symptom(p2,shortbreath)",
    "has_symptom(p2,blockednose)",
    "has_symptom(p3,fever)",
    "has_symptom(p3,headache)"
])


print(respdis.query(pl.Expr("has_symptom(X,Y)")))

[{'X': 'p1', 'Y': 'cough'}, {'X': 'p1', 'Y': 'fever'}, {'X': 'p1', 'Y': 'shortbreath'}, {'X': 'p2', 'Y': 'shortbreath'}, {'X': 'p2', 'Y': 'blockednose'}, {'X': 'p3', 'Y': 'fever'}, {'X': 'p3', 'Y': 'headache'}]


### We query our knowledge base for conditions that match the symptoms

What conditions might our patients have?  


In [28]:
respdis(["has_condition(X,D) :- has_symptom(X,Y), symptom(Y,D)"])

print(respdis.query(pl.Expr("has_condition(X,D)")))


conditions = set([ (list(y.values())[0],list(y.values())[1]) for y in respdis.query(pl.Expr("has_condition(X,D)")) ])

has_condition = dict()

for (p,c) in conditions:
    if p not in has_condition:
        has_condition[p]=[]
    if c not in has_condition[p]:
        has_condition[p].append(c)
        
has_condition

[{'X': 'p1', 'D': 'covid'}, {'X': 'p1', 'D': 'flu'}, {'X': 'p1', 'D': 'covid'}, {'X': 'p1', 'D': 'flu'}, {'X': 'p1', 'D': 'covid'}, {'X': 'p1', 'D': 'allergy'}, {'X': 'p2', 'D': 'covid'}, {'X': 'p2', 'D': 'allergy'}, {'X': 'p2', 'D': 'cold'}, {'X': 'p2', 'D': 'allergy'}, {'X': 'p3', 'D': 'covid'}, {'X': 'p3', 'D': 'flu'}, {'X': 'p3', 'D': 'flu'}, {'X': 'p3', 'D': 'covid'}, {'X': 'p3', 'D': 'cold'}]


{'p1': ['flu', 'allergy', 'covid'],
 'p2': ['covid', 'cold', 'allergy'],
 'p3': ['covid', 'flu', 'cold']}

### Now we exclude conditions for which symptoms are not present

Previously we could see any condition that the patient might have based on any symptom that the patient might have. Now we try to force the system to give us only the best recommendations by taking absent symptoms into account as well. 

- fever is rare in colds and allergies
- headache is not present in allergies
- cough is seldom present in colds and never in allergies
- shortness of breath is not present in colds and flu

In [29]:
respdis(["not_symptom(fever,cold)",
    "not_symptom(fever,allergy)",
    "not_symptom(headache,allergy)",
    "not_symptom(blockednose,covid)",
    "not_symptom(blockednose,flu)",
    "not_symptom(cough,cold)",
    "not_symptom(cough,allergy)",
    "not_symptom(shortbreath,cold)",
    "not_symptom(shortbreath,flu)",
    "not_condition(X,D) :- has_symptom(X,Y) , not_symptom(Y,D)"])

not_conditions = set([ (list(y.values())[0],list(y.values())[1]) for y in respdis.query(pl.Expr("not_condition(X,D)")) ])

print(not_conditions)

not_has_condition = dict()

for (p,c) in not_conditions:
    if p not in not_has_condition:
        not_has_condition[p]=[]
    if c not in not_has_condition[p]:
        not_has_condition[p].append(c)
        
only_conditions = dict()

for p in has_condition.keys():
    conds = has_condition[p]
    not_conds = not_has_condition[p]
    only_conditions[p]=[x for x in conds if x not in not_conds]
    

print(respdis.query(pl.Expr("has_symptom(X,Y)")) )

    
only_conditions

{('p1', 'flu'), ('p1', 'cold'), ('p2', 'covid'), ('p1', 'allergy'), ('p2', 'flu'), ('p2', 'cold'), ('p3', 'cold'), ('p3', 'allergy')}
[{'X': 'p1', 'Y': 'cough'}, {'X': 'p1', 'Y': 'fever'}, {'X': 'p1', 'Y': 'shortbreath'}, {'X': 'p2', 'Y': 'shortbreath'}, {'X': 'p2', 'Y': 'blockednose'}, {'X': 'p3', 'Y': 'fever'}, {'X': 'p3', 'Y': 'headache'}]


{'p1': ['covid'], 'p2': ['allergy'], 'p3': ['covid', 'flu']}